In [1]:
import itertools
import numpy as np
from collections import defaultdict

In [2]:
def read_file(path = 'YandexRelPredChallenge.txt'):
    yandex_log = []
    with open(path, 'r') as f:
        yandex_log = f.readlines()

    yandex_log_ = []
    for line in yandex_log:
        line = line.strip()
        yandex_log_.append(line.split('\t'))
        
    return yandex_log_

In [3]:
# by Thanos Efthymiou

def queries_pickle(file_lines):    
    
    queries= {}
    last_session = -1
    
    for line in file_lines:
        
        if line[0] != last_session:
            last_session = line[0]
            session_queries = []
        
        
        if line[2] =='Q':
            
            session_queries.append(line[3])
            
            if line[3] not in queries:
                
                queries[line[3]] = []
                queries[line[3]].append([line[5:],[]])
                
            else:
                
                queries[line[3]].append([line[5:],[]])
                
            
        else:
    
            
            if line[3] in queries[session_queries[-1]][-1][0]:
                
                queries[session_queries[-1]][-1][1].append(line[3])
                
            else:
                
                for sess_q in reversed(session_queries):
                    if line[3] in queries[sess_q][-1][0]:
                        queries[sess_q][-1][1].append(line[3])
                        break
                    
    return queries
        
        
        
def queries_clicks(queries):
    
    
    query_doc_clicks = {}


    for query in queries.keys():
        query_doc_clicks[query] = []
        for docs in queries[query]:
            
            doc_clicks = [0]*10
            for click in docs[1]:
            
                doc_clicks[docs[0].index(click)] = 1
            
            query_doc_clicks[query].append(doc_clicks)
   
    return query_doc_clicks

## Load Data (use dataset from IR1)

In [4]:
path = '/media/georgios/LENOVO/gnsid/Documents/MSc/IR2/YandexRelPredChallenge.txt'

lines = read_file(path)

# use both variables as global
queries_dict = queries_pickle(lines)
query_doc_clicks = queries_clicks(queries_dict)

In [5]:
for q in queries_dict['275']:
    print (q)

[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2536', '2506']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '1606', '2524']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '1606']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '2510', '2536']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '1606']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '1606', '1606']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], []]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506', '1606', '2536']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519'], ['2506']]
[['2506', '1606', '2536', '2512', '2524', '2510', '2513', '2509', '2526', '2519']

In [6]:
for c in query_doc_clicks['275']:
    print (c)

[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Get Document Representation

In [7]:
n = 10

# use it as global variable
# all possible click combinations of the 10 documents in a SERP
lst = list(itertools.product([0, 1], repeat=n))

In [8]:
def serp2d(serp_clicks, d_pos):
    """
    serp_clicks: serp list with zeros and ones if the doc at the respective position was clicked,
    d_pos: the position of the document for which we are making the representation

    Output: representation of document d with a vector of size of 10240.
    """

    # init representation
    d = [0]*10240

    clicks = tuple(serp_clicks)

    index = lst.index(clicks) 
    pos = index + d_pos * 1024

    d[pos] = 1

    return np.array(d)

In [9]:
def get_doc_repres(query_id, doc_id):
    doc_repr = np.zeros(10240)
    if query_id in queries_dict.keys():

        for index, serp_and_clicked_docs in enumerate(queries_dict[query_id]):
            serp = serp_and_clicked_docs[0]
            if doc_id in serp:
                serp_clicks = query_doc_clicks[query_id][index]
                doc_pos = serp.index(doc_id)
                #print(serp2d(serp_clicks, doc_pos).index(1))
                doc_repr += serp2d(serp_clicks, doc_pos)
    
    return doc_repr

In [10]:
query_id = '275'
doc_id = '2506'
d_r = get_doc_repres(query_id, doc_id)

In [11]:
d_r[512]

4.0